In [3]:
# pip install ipykernal
# pip install ipkernel
pip install requests

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3156794279.py, line 3)

In [ ]:
import numpy as np 
import pandas as pd 
import requests as api

class WaterApi:


    def __init__(self) -> None:
        URL  = "http://nodewater.whf.bz/json.php?apikey=LIFQpwnVeyZ84Yvxk9OPDAaXV9PsbIGc&view=true"
        res = api.get(URL)
        __data =  res.json()

        self.data_set = pd.DataFrame(__data['Data'])

        self.data_set =  self.data_set.dropna() if self.data_set.iloc[-1].isnull().sum()== 0 else self.data_set[:-1].dropna()

        self.data_set['start_time'] = pd.to_datetime(self.data_set['start_time'] )

        # Make a copy of the DataFrame
        self.data_new = self.data_set.copy()

        # Set 'start_time' as the index
        self.data_new .set_index('start_time', inplace=True)

    
    # def filt (self):
    #     self.data_set['start_time'] = pd.to_datetime(self.data_set['start_time'] )

    #     # Make a copy of the DataFrame
    #     self.data_new = self.data_set.copy()

    #     # Set 'start_time' as the index
    #     self.data_new .set_index('start_time', inplace=True)
    #     return self.data_new


    def __get_data_by_filter ( self  , date_flag , value):
            return self.data_new.resample(date_flag)[value ] 

        # monthly_duration = df1.resample('M')['duration' ] 
    def get_monthly(self) :
        monthly_duration =  self.__get_data_by_filter('M' , 'duration')
        monthly_litre_water =  self.__get_data_by_filter('M' , 'litreWater')
        # print(type(monthly_duration))
        print("Total Duration of motor run")

        m_d = monthly_duration.sum().reset_index('start_time')
        # print(m_d)

        # print("Total Litre of Water has used :")
        m_l = monthly_litre_water.sum().reset_index('start_time')
        # print(m_l)

        return pd.merge(m_d , m_l  , on='start_time'  , how="inner")

    def get_weekly(self):
        weekly_duration = self.__get_data_by_filter('W' , 'duration')
        weekly_water_litre = self.__get_data_by_filter('W' , 'litreWater')

        w_d = weekly_duration.sum().reset_index('start_time')
        w_l = weekly_water_litre.sum().reset_index('start_time')


        return pd.merge(w_d, w_l , on="start_time" , how="inner")
    
    def get_daily (self):
        daily_duration = self.__get_data_by_filter('D' , 'duration')
        daily_litre_water = self.__get_data_by_filter('D' , 'litreWater')

        d_d = daily_duration.sum().reset_index('start_time')
        d_l = daily_litre_water.sum().reset_index('start_time')

        return  pd.merge(d_d , d_l , on="start_time" , how="inner")
         

# test 

d = WaterApi()
print(d.data_set)


ValueError: time data "13-01-2024 02:55" doesn't match format "%m-%d-%Y %H:%M", at position 3. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [9]:
d.data_set.count()

id            1197
machine_id    1197
start_time    1197
end_time      1197
duration      1197
litreWater    1197
dtype: int64

In [12]:
d.get_daily()

AttributeError: 'WaterApi' object has no attribute 'data_new'

In [44]:
import numpy as np
import pandas as pd
import requests as api

class WaterApi:
    def __init__(self) -> None:
        URL = "http://nodewater.whf.bz/json.php?apikey=LIFQpwnVeyZ84Yvxk9OPDAaXV9PsbIGc&view=true"
        res = api.get(URL)
        __data = res.json()

        # Load data into a DataFrame
        self.data_set = pd.DataFrame(__data['Data'])
        self.data_set['duration'] = self.data_set['duration'].astype(int)
        self.data_set['litreWater'] = self.data_set['litreWater'].astype(float)

        # Drop missing values if the last row is completely null
        self.data_set = self.data_set.dropna() if self.data_set.iloc[-1].isnull().sum() == 0 else self.data_set[:-1].dropna()

        # Convert 'start_time' to datetime with the correct format
        self.data_set['start_time'] = pd.to_datetime(self.data_set['start_time'], format="%d-%m-%Y %H:%M")

        # Make a copy of the DataFrame
        self.data_new = self.data_set.copy()

        # Set 'start_time' as the index
        self.data_new.set_index('start_time', inplace=True)

    def __get_data_by_filter(self, date_flag, value):
        return self.data_new.resample(date_flag)[value]

    def get_monthly(self):
        monthly_duration = self.__get_data_by_filter('M', 'duration')
        monthly_litre_water = self.__get_data_by_filter('M', 'litreWater')
        print("Total Duration of motor run")

        m_d = monthly_duration.sum().reset_index('start_time')
        m_l = monthly_litre_water.sum().reset_index('start_time')

        return pd.merge(m_d, m_l, on='start_time', how="inner")

    def get_weekly(self):
        weekly_duration = self.__get_data_by_filter('W', 'duration')
        weekly_water_litre = self.__get_data_by_filter('W', 'litreWater')

        w_d = weekly_duration.sum().reset_index('start_time')
        w_l = weekly_water_litre.sum().reset_index('start_time')

        return pd.merge(w_d, w_l, on="start_time", how="inner")

    def get_daily(self):
        daily_duration = self.__get_data_by_filter('D', 'duration')
        daily_litre_water = self.__get_data_by_filter('D', 'litreWater')

        d_d = daily_duration.sum().reset_index('start_time')
        d_l = daily_litre_water.sum().reset_index('start_time')

        return pd.merge(d_d, d_l, on="start_time", how="inner")
       


# Test
d = WaterApi()
print(d.data_set.head(3))


  id  machine_id          start_time          end_time  duration  litreWater
0  1  rf6xuvTNAp 2024-01-12 06:57:00  12-01-2024 06:59       174    4.569732
1  2  rf6xuvTNAp 2024-01-12 13:34:00  12-01-2024 13:36       132    3.466694
2  3  rf6xuvTNAp 2024-01-12 22:02:00  12-01-2024 22:05       161    4.228316


In [46]:
# d.get_daily().head(10)
d.get_daily()

,start_time,duration,litreWater
0,2024-01-12,467,12.264742
1,2024-01-13,724,19.014289
2,2024-01-14,761,19.986014
3,2024-01-15,640,16.808211
4,2024-01-16,472,12.396056
...,...,...,...
291,2024-10-29,538,14.129403
292,2024-10-30,438,11.503120
293,2024-10-31,319,8.377843
294,2024-11-01,637,16.729423


In [26]:
d.get_monthly().head(10)

Total Duration of motor run


C:\Users\haris\AppData\Local\Temp\ipykernel_10936\2826220214.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return self.data_new.resample(date_flag)[value]


,start_time,duration,litreWater
0,2024-01-31,1741321611311781381441331371451711801281391521...,4.569732423.466693564.228315633.440430734.6747...
1,2024-02-29,1551461661291791261331801351671461501251341341...,4.070738653.834373184.359629783.387905074.7010...
2,2024-03-31,2032972922072322582972402222551812892491992362...,5.331354497.800060517.668746365.436405816.0929...
3,2024-04-30,2572122192822702652052562552192462661822342642...,6.749547315.567719965.751559777.406118067.0909...
4,2024-05-31,2892252212792992952962772232772922562102082682...,7.589957875.909136755.804085437.327329577.8525...
5,2024-06-30,1952562002751872312692792432182572082192822122...,5.121251856.723284485.2525667.222278254.911149...
6,2024-07-31,1341126689153116801511716514366681607613211516...,3.519219222.941436961.733346782.337391874.0182...
7,2024-08-31,9314382168135168961231381431101261331131001031...,2.442443193.755584692.153552064.412155443.5454...
8,2024-09-30,1239590126122941031771401431771626114564897167...,3.230328092.494968852.36365473.309116583.20406...
9,2024-10-31,1191771048210812887128118111147146173168109113...,3.125276774.648520912.731334322.153552062.8363...


In [39]:
d.data_new.head(3)

,id,machine_id,end_time,duration,litreWater
start_time,,,,,
2024-01-12 06:57:00,1,rf6xuvTNAp,12-01-2024 06:59,174,4.56973242
2024-01-12 13:34:00,2,rf6xuvTNAp,12-01-2024 13:36,132,3.46669356
2024-01-12 22:02:00,3,rf6xuvTNAp,12-01-2024 22:05,161,4.22831563


In [43]:
d.data_new.dtypes
d.data_new['litreWater' ] =  d.data_new['litreWater' ].astype(float)
d.data_new.dtypes


id             object
machine_id     object
end_time       object
duration       object
litreWater    float64
dtype: object

In [41]:
d.data_set.dtypes

id                    object
machine_id            object
start_time    datetime64[ns]
end_time              object
duration              object
litreWater            object
dtype: object